<a href="https://colab.research.google.com/github/Hiba-Chaabnia/Finance-Chatbot-Budget-Planner/blob/main/Multi_Agent_Budget_Planner_with_Built_in_Financial_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Pw1NIj8AzjlUmy0hVL2nSR1VidMy-MG_?authuser=1#scrollTo=UzSZdG1cYkwh)


# Multi-Agent Budget Planner with Built-in Financial Chatbot

### **Project Overview**
This project implements an intelligent budget planning system that leverages multiple AI agents to provide personalized financial guidance. Built using the `CrewAI` framework and powered by `Google's Gemini 1.5 Flash` model, the system creates a collaborative environment where specialized AI agents work together to deliver comprehensive financial planning services.

### **System Architecture**
The system consists of four specialized AI agents:
1. **Financial Interviewer (Chatbot)**: Simulates user interactions to test data collection processes
2. **Information Extractor**: Processes and structures financial data
3. **Financial Analyst**: Creates personalized budget plans
4. **Financial Advisor**: Provides actionable financial recommendations

![Multi-Agent Budget Planner with Built-in Financial Chatbot System Architecture](https://drive.google.com/uc?export=view&id=1mCxsyuHsvzL2zFqc0uHMZTf3Pc9xh5JJ)



##01. Envirorment Setup

⚠️ CrewAI requires Python version >=3.10 and <=3.13


In [2]:
import sys
print(sys.version)

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]


📦 Installing the necessary libraries for the project

In [3]:
!pip install crewai crewai_tools langchain &>/dev/null

## 02. Importing Libraries

In [4]:
from crewai import Agent, Task, Crew, LLM

🔒 Retrieving the Gemini API from the environment variable, follow these step-by-step instructions:

1. Generate the API Key:
   - If you haven’t created an API key yet, visit [aistudio.google.com](https://aistudio.google.com) to generate one.

2. Add the API Key in Google Colab:
   - In your Google Colab notebook, click on the key (secret) icon 🔑 on the left sidebar.
   - Click on the "Add new secret" button.
   - Name the secret `GEMINI_API_KEY`.
   - Paste your generated API key into the Value field.
   - Toggle the Notebook Access.

In [5]:
from google.colab import userdata
GEMINI_API_KEY=userdata.get('GEMINI_API_KEY')

🦜 Initializing the Gemini 1.5 Flash model

In [6]:
gemini_model = LLM (model='gemini/gemini-1.5-flash',
                    temperature=0.7,
                    api_key= GEMINI_API_KEY)

In [7]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## 03. Creating Agents

In this step, we'll create instances of the `Agent` class, each with specific properties that define their role, goal, and backstory:

- **`role`**: The agent's function within the crew 🛠️
- **`goal`**: The agent's objective, guiding their decision-making 🎯
- **`backstory`**: Provides context to the agent's role and goal, enhancing interactions and collaboration 📖

### Agent 1 : Financial Interviewer 💬💰


In [24]:
Financial_Interviewer = Agent(
    role="Chatbot Interaction Designer specialized in Personal Finance",

    goal="Create an engaging, user-friendly chatbot to collect detailed "
         "financial information from users, including monthly income,"
         "fixed expenses, discretionary spending, and savings goals."
         "Ensure all questions are clear and responses are properly validated."
         "If there is no user input, generate one yourself",

    backstory=(
        "As a Chatbot Interaction Designer, you have previously worked on "
        "developing customer-facing chatbots for banks and personal finance apps."
        "You excel at understanding user pain points and crafting questions "
        "that make financial less intimidating. Your experience with"
        "handling sensitive data ensures that the system is secure "
        "and compliant with privacy regulations. "),

    llm = gemini_model,
    verbose = True,
    memory=True
)

### Agent 2: Information Extractor 💼📊

In [25]:
Information_Extractor = Agent(
    role="Data Engineer for Financial Systems",

    goal="Process and structure user inputs into"
         "categorized data, identifying gaps or inconsistencies."
         "Ensure accuracy and completeness for subsequent calculations.",

    backstory=(
        "You have a strong background in financial data "
        "processing, having previously worked on systems that categorize and "
        "validate transactions for budgeting apps like Mint. Your expertise "
        "in data pipelines and natural language processing makes you adept at "
        "handling unstructured text and converting it into actionable information."
    ),

    llm = gemini_model,
    allow_delegration=False,
    verbose=True,
    memory=True
)

### Agent 3: Financial Analyst 💰📈

In [26]:
Financial_Analyst = Agent(
    role="Financial Analyst Specializing in Budget Allocation and Optimization.",
    goal="Create a personalized budget plan that aligns with the user's "
         "needs, wants, and financial aspirations."
         "Utilize the 50/30/20 rule as a foundation. ",

    backstory=(
        "With a vast knowledge of financial planning principles, "
        "budgeting methodologies, and investment strategies, you excel at "
        "analyzing complex financial scenarios, identifying potential risks "
        "and rewards, and recommend optimal budget allocations"
        "to help users achieve their financial goals."),

    llm = gemini_model,
    allow_delegration=False,
    verbose=True,
)

### Agent 4: Financial Advisor 💡💵

In [27]:
Financial_Advisor = Agent(
    role="Financial Advisor Specializing in "
         "Personalized Financial Guidance and Education.",

    goal="Empower users to make informed financial choices "
         "and achieve their financial goals by providing them "
         "with actionable financial advice and personalized recommendations",

    backstory=(
        "As a compassionate and knowledgeable financial advisor, you"
        "are passionate about helping others improve their financial well-being. "
        "You possess excellent communication and interpersonal skills, "
        " enabling you to build strong relationships with users "
        "and provide clear, actionable advice."),

    llm = gemini_model,
    allow_delegration=False,
    verbose=True,
)

## 04. Creating Tasks

Now, it's time to define the **tasks**! Tasks are specific assignments for agents and provide all the details they need to execute them successfully.

Each task includes:

- **`description`**: What the task is all about 📝
- **`expected_output`**: What the outcome should look like 🎯
- **`agent`**: The agent responsible for completing the task 🤖

### Task for Financial Interviewer Agent: Collect Financial Information

In [28]:
collect_financial_info = Task(

    description=(
        "Collect detailed financial information from the user. Ask questions about:"
        "1. Monthly income (salary, freelance earnings, rental income, etc.)"
        "2. Fixed expenses (rent/mortgage, utilities, insurance, etc.)"
        "3. Discretionary spending (dining out, entertainment, hobbies, etc.)"
        "4. Savings goals (emergency fund, retirement, vacation, etc.)"
        "Ensure all responses are validated for completeness and accuracy."
    ),

    expected_output=(
        "A structured summary including:"
        "1. Income details (e.g., salary, freelance earnings)."
        "2. Expense breakdown (e.g., rent, groceries)."
        "3. Savings and investments."
        "4. Financial goals and timelines."),

    agent=Financial_Interviewer,

)

### Task for Information Extractor Agent: Process Financial Data

- A **list of tasks** can be passed as `context` to another task 🔗
- The task will **consider the outputs** of the tasks in the list during its execution 🛠️
- The task will **not run** until it has received the necessary output(s) from those tasks ⏳

In [29]:
process_financial_data = Task(
    description=(
        "Process and structure the collected financial information. Categorize data into:"
        "1. Income sources"
        "2. Fixed expenses"
        "3. Discretionary spending"
        "4. Savings goals"
        "Identify any gaps or inconsistencies in the data and request clarification if needed."
    ),

    expected_output=(
        "The output should provide the following structured details:"
        "1. A categorized list of income sources (e.g., salary, freelance, rental income)."
        "2. Fixed expenses categorized by type (e.g., rent, utilities, insurance)."
        "3. Variable expenses categorized by type (e.g., groceries, entertainment)."
        "4. Details of current savings and investments (e.g., bank savings, stocks, bonds)."
        "5. A summary highlighting inconsistencies, missing data, or anomalies in the financial information."
        ),

    context= [collect_financial_info],
    agent= Information_Extractor ,

)

### Task for Financial Analyst Agent: Create Budget Plan

In [30]:
create_budget_plan = Task(
    description=(
        "Create a personalized budget plan based on the processed data. Utilize the 50/30/20 rule:"
        "1. Allocate 50% of income to needs (fixed expenses)"
        "2. Allocate 30% of income to wants (discretionary spending)"
        "3. Allocate 20% of income to savings and debt repayment"
        "Adjust allocations based on the user's specific goals and financial situation."),

    expected_output=(
        "The output should include: "
        "- A detailed budget plan with allocations as per the 50/30/20 rule. "
        "- Needs: 50% of income allocated to fixed expenses. "
        "- Wants: 30% of income allocated to discretionary spending. "
        "- Savings: 20% of income allocated for savings and investments. "
        "- A summary highlighting any adjustments needed."
    ),

    context=[process_financial_data],
    agent= Financial_Analyst
)

### Task for Financial Advisor Agent: Provide Financial Advice

In [31]:
provide_financial_advice = Task(
    description=(
        "Provide actionable financial advice and recommendations. Include"
        "tips on reducing fixed expenses, managing discretionary spending, "
        "achieveing savings goals, and investment strategies for "
        "long-term financial health. Ensure the advice"
        "is personalized and aligns with the user's financial aspirations."
    ),

    expected_output=(
        "Personalized advice and steps for improving financial"
        "well-being."
    ),

    context=[create_budget_plan],
    agent= Financial_Advisor
)


## 05. Forming Crew

The **crew** is a group of AI agents working together to complete a set of tasks.

A crew is defined by:

- **`tasks`**: A list of tasks to be performed 📝
- **`agents`**: A list of the agents involved 🤖
- **`verbose=True`**: Allows you to see all execution logs for full transparency 🖥️

In [32]:
budget_planner_crew  = Crew(
    agents=[Financial_Interviewer, Information_Extractor, Financial_Analyst, Financial_Advisor],
    tasks=[collect_financial_info, process_financial_data, create_budget_plan, provide_financial_advice],
    verbose=True
)

### Kicking off the crew

⏰ This execution will take a few minutes to run

In [33]:
result = budget_planner_crew.kickoff()

# Agent: Chatbot Interaction Designer specialized in Personal Finance
## Task: Collect detailed financial information from the user. Ask questions about:1. Monthly income (salary, freelance earnings, rental income, etc.)2. Fixed expenses (rent/mortgage, utilities, insurance, etc.)3. Discretionary spending (dining out, entertainment, hobbies, etc.)4. Savings goals (emergency fund, retirement, vacation, etc.)Ensure all responses are validated for completeness and accuracy.


# Agent: Chatbot Interaction Designer specialized in Personal Finance
## Final Answer: 
Hi there! Let's get a clearer picture of your finances.  Don't worry, this is completely confidential and will help us tailor advice to your specific situation.  We'll start with your income.

**1. Income:**

* **What is your approximate monthly salary or wage from your primary employment?** (Please enter a numerical value, e.g., 3000)
* *Validation:* Checks if the input is a number and greater than 0. If not, prompts: "Please ent

In [34]:
from IPython.display import Markdown

result_text = result.raw
Markdown(result_text)

Let's work together to create a personalized financial plan that helps you achieve your financial goals.  The budget you've provided is a great starting point, but we need to refine it based on your priorities and make it truly actionable.


**Addressing the Budget Imbalance:**

Your current budget shows that your "Needs" exceed the 50% allocation of the 50/30/20 rule. This requires immediate attention. We can address this through a combination of strategies:

**1. Reducing Fixed Expenses (Option 1):**

Let's analyze your fixed expenses:

* **Housing ($1000):** This is a significant portion of your budget.  Can you consider more affordable housing options? Could you potentially negotiate a lower rent?  If you own your home, explore ways to lower your mortgage payments (refinancing, etc.).

* **Utilities ($200):** Can you reduce your energy consumption (using energy-efficient appliances, turning off lights)?  Shop around for better internet and phone plans.

* **Insurance ($100):**  Review your insurance policies. Are you getting the best coverage at the best price?  Consider increasing your deductible to lower your premiums.  Compare quotes from multiple insurance providers.

* **Other Fixed Expenses ($50):** Identify these expenses. Are any subscriptions unnecessary? Can you cancel or downgrade any services?

**Actionable Steps:**

1. **Detailed Expense Tracking:** For the next month, meticulously track every expense. Categorize them (needs, wants, savings).  Use budgeting apps or spreadsheets. This will give us a clear picture of your spending habits.

2. **Negotiate:** Contact your service providers (internet, phone, insurance) and negotiate lower rates.  Often, simply asking can lead to savings.

3. **Re-evaluate Housing:** If possible, explore alternative housing options that better align with your budget.

4. **Subscription Audit:** Cancel or downgrade any unnecessary subscriptions (streaming services, gym memberships, etc.).


**2. Increasing Income (Option 2):**

While reducing expenses is crucial, increasing your income can provide more financial flexibility.

**Actionable Steps:**

1. **Identify Skills:** What skills or talents do you possess that you could monetize?  Consider freelancing, gig work, or part-time employment.

2. **Explore Opportunities:** Research part-time jobs or freelance platforms that align with your skills and availability.

3. **Negotiate a Raise:** If you're satisfied with your current job, research industry standards for your role and experience level and prepare a case for a salary increase.


**Revised Budget and Savings Allocation (Illustrative):**

Let's assume, after implementing the above strategies, you reduce your "Needs" to $1200.  This leaves $1300 for "Wants" and "Savings & Debt Repayment."  Applying the 50/30/20 rule, we'd allocate approximately $390 to savings and debt repayment.

* **Emergency Fund:**  We'll allocate a significant portion towards building a $5000 emergency fund.  Given the current situation, let's allocate $200 monthly.  This will take roughly 25 months to achieve.

* **Down Payment:**  The remaining $190 will be allocated towards your $20,000 down payment goal.  This will take approximately 8.5 years.  This timeline may need adjusting based on your priorities and ability to increase savings.

**Investment Strategies for Long-Term Financial Health:**

Once you have a comfortable emergency fund (3-6 months of living expenses), it's crucial to start investing for long-term growth.

* **Retirement Savings:** Contribute consistently to a retirement account (401k, IRA).  Take advantage of employer matching if available.

* **Investment Portfolio:**  Diversify your investments across different asset classes (stocks, bonds, real estate). Consider low-cost index funds or ETFs for diversification.

* **Regular Contributions:**  Automate your investments to ensure consistent contributions.

**Ongoing Monitoring and Review:**

This is not a one-time plan.  Regularly review your budget (monthly or quarterly) to track your progress, make adjustments as needed, and celebrate your successes.


This detailed plan provides actionable steps and a framework for improving your financial well-being. Remember, consistency and discipline are key to achieving your financial goals.  Let's schedule a follow-up to discuss your progress and refine the plan further.